<h5> Αρχικά εισάγεται η βιβλιοθήκη mowl και η torch
και αρχικοποιείται η διαθέσιμη προς χρήση μνήμη για το Java Virtual Machine ως 10 Gigabytes.

**Προσοχή: πριν να τρέξει ο κώδικας του github θα πρέπει να έχει εγκατασταθεί στο virtual environment το πακέτο της βιβλιοθήκης mOWL (!pip install mowl-borg). Αυτή η διαδικασία γίνεται "χειροκίνητα" από τον κάθε χρήστη που εκτελεί το collab.** </h5>

In [1]:
!pip install mowl-borg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.4/718.4 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.1 MB/s eta 0:00:00
  Created wheel for temp: filename=temp-2020.7.2-py3-none-any.whl size=1230 sha256=5b89d30e24dc852d82516ee028237ac180879bb19a7f44bcf47d70b226cc89b1
  Stored in directory: /root/.cache/pip/wheels/54/f4/e0/661461cb3cbc2cd950cdd0ce8ba52bdc52a1122784a2f0e1dc
Successfully built temp


In [ ]:
import mowl
mowl.init_jvm("10g")
import torch as th

<h3>Dataset που χρησημοποιήθηκε:</h3>
Το dataset που χρησημοποιήθηκε είναι το PPIYeastDataset το οποίο είναι built-in στην mowl και οι σχέσεις μεταξύ πρωτεϊνών αναπαραστούνται σαν αξιώματα με τον παρακάτω τρόπο: <br>
<b>protein1 ⊆ interacts_with.protein2</b> <br>
Η επιλογή dataset επηρεάζει σημαντικά τον κώδικα καθώς αλλάζουν οι μέθοδοι για training και evaluation αναλόγως με την αρχιτεκτονική του dataset.



<h3>EL-Embedding μοντέλο</h3>
Η EL γλώσσα απαρτίζεται απο τα παρακάτω GCIs:

**GCI 0** -`C ⊑ D` <br>
**GCI 1** -`C1 ⊓ C2 ⊑ D`<br>
**GCI 2**-`C ⊑ ∃R.D` <br>
**GCI 3**-`∃R.C ⊑ D` <br>
**GCI bot 0**-`C ⊑ ⊥` <br>
**GCI bot 1**-`C1 ⊓ C2 ⊑ ⊥` <br>
**GCI bot 3**- `∃R.C ⊑ ⊥` <br>

Στο EL module, η βασική λειτουργία είναι να οριστεί loss function για κάθε ένα από τα GCI.  Για τα PPI, υπάρχει έτοιμο model το ELEmPPI, αλλά σε διαφορετικά datasets, η επεξεργασία της κλάσης είναι αρκετά απλή. <br>(https://mowl.readthedocs.io/en/latest/_modules/mowl/models/elembeddings/examples/model_ppi.html#ELEmPPI) <br>



## Εκπέδευη μοντέλου EL embedding
Στις παραμέτρους του ELEmPPI μπορούν να δοκιμαστούν διάφορες τιμές για κάθε όρισμα

In [ ]:
import logging
from mowl.datasets.builtin import PPIYeastDataset
from mowl.models import ELEmPPI

# logging να αγνοεί τα info μηνύματα
logging.basicConfig(level=logging.WARNING)

# dataset που χρησιμοποιείται (το συγκεκριμένο είναι built-in)
dataset = PPIYeastDataset()

# initialize model with the required parameters
model = ELEmPPI(dataset, #το dataset που χρησιμοποιείται για εκπέδευση (στο συγκεκριμένο μέσα στο ELemPPI παίρνει το training κομμ΄άτι του και τα GCIs αυτού
                embed_dim=30, #διάσταση των embedding
                learning_rate=0.001,#ρυθμός μάθησης (ρυθμός ενημέρωσης παραμέτρων εκπέδευσης σε κάθε βήμα)
                epochs=100,#αριθμός εποχών (δλδ πλήρης επαναλήψεων εκπέδευσης)
                batch_size=20000, #τα δείγματα που επεξεργάζονται σε κάθε βήμα
                model_filepath=None,
                device='cuda') #Χρήση cuda (gpu) για γρηγορότερους υπολογισμούς στο training

#κλήση μεθόδου εκπέδευσης
model.train()


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:14<00:00,  1.34it/s]


1

## Evaluating

To evaluation των embeddings διαφέρει ανάλογα με το dataset. Στο συγκεκριμένο παράδειγμα, χρησιμοποιείται το PPIYeastDataset και το built-in PPIEvaluator. <b> Κάθε κλάση Evaluator χαρακτηρίζεται από δύο κύρια σημεία: τον τύπο των οντοτήτων που εμπλέκονται και τον τύπο των αξιωμάτων που θα αξιολογηθούν. </b> Στην περίπτωση του PPIEvaluator, οι εμπλεκόμενες οντότητες είναι μόνο αυτές που εκπροσωπούν πρωτεΐνες, χωρίς να λαμβάνονται υπόψη άλλες οντότητες. Τα αξιώματα που αξιολογούνται με τον PPIEvaluator είναι του τύπου "p1 αλληλεπιδρά με p2" (εξού και το PPI όνομα). Κάθε dataset έχει τη δική του ιδιότητα evaluation_classes, όπου καθορίζει τι είδους οντότητες ή αξιώματα θα εξεταστούν από το dataset στο πλαίσιο της αξιολόγησης.

In [ ]:
from mowl.evaluation import PPIEvaluator
model.set_evaluator(PPIEvaluator)
model.evaluate(dataset.testing)

data1 = model.metrics #αποθήκευση αποτελεσμάτων για το ELembedding

## ELBox Embedding
Στην περίπτωση των EL Embeddings, τα γεωμετρικά αντικείμενα που αναπαριστούν τις κλάσεις της οντολογίας είναι μπάλες n-διαστάσεων. Ένα από τα GCI στην EL είναι: <br>
`C1 ⊓ C2 ⊑ D'
Όπως είναι φανερό, σε αυτό το GCI, υπάρχει μια πράξη τομής . Η υπολογιστική αυτή τομή χρησιμοποιώντας μπάλες δεν είναι μια κλειστή πράξη διότι η περιοχή που περιέχεται στην τομή δύο μπαλών δεν είναι μπάλα. Για να λυθεί αυτό το ζήτημα, γίνεται αλλαγή γεωμετρικών αντικειμένων σε κουτιά, για τα οποία η πράξη της τομής έχει την ιδιότητα της κλειστότητας.


In [ ]:
from mowl.models.elboxembeddings.examples.model_ppi import ELBoxPPI
model = ELBoxPPI(dataset,
                 embed_dim=30,
                 learning_rate=0.001,
                 epochs=100,
                 batch_size=20000,
                 model_filepath=None,
                 device='cuda')
model.train()

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:44<00:00,  2.24it/s]


1

Εδώ χρησιμοποιείται για άλλη μια φορά το built-in evaluator για PPIs της mowl και αποθηκεύονται τα αποτελέσματα απόδοσης του embedding.

In [ ]:
from mowl.evaluation import PPIEvaluator

model.set_evaluator(PPIEvaluator)
model.evaluate(dataset.testing)
data2= model.metrics

## Embedding με χρήση γράφου
<h3>TransE Embedding</h3>
Στο παράδειγμα αυτό αρχικά προβάλλονται DL αξιώματα με του dataset σε γράφο με χρήση του DL2VecProjector και στην συνέχεια χρησιμοποιείται το μοντέλο TransE, όπου συνδυάζει τα διανύσματα των οντοτήτων με τις σχέσεις ως διανυσματικές μεταφράσεις, έτσι ώστε η αρχική οντότητα συν την σχέση να προσεγγίζει την τελική οντότητα (h + r ≈ t), για την παραγωγή των embeddings.

In [ ]:
from mowl.models import GraphPlusPyKEENModel
from mowl.projection import DL2VecProjector
from pykeen.models import TransE

model = GraphPlusPyKEENModel(dataset,device='cuda')
model.set_projector(DL2VecProjector()) #προβολή του dataset σε γράφο
model.set_kge_method(TransE, random_seed=42) #χρηση του transE για την παραγωγη embedding στον γράφο γνώσης
model.optimizer = th.optim.Adam
model.lr = 0.001
model.batch_size = 32

model.train(epochs = 20) #εκπέδευση μοντελου





There are 6840 classes not found in the graph. They might be ignored in the projection or they might be in the validation/testing set but not in the training set.


Training epochs on cuda:0:   0%|          | 0/20 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/10615 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/10615 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/10615 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0/10615 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/10615 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/10615 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/10615 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/10615 [00:00<?, ?batch/s]

Evaluation μοντέλου με χρήση του PPIEvaluator και αποθήκευση μετρικών απόδοσης.

In [ ]:
model.set_evaluator(PPIEvaluator)
model.evaluate(dataset.testing)
data3= model.metrics

A custom EvaluationModel should be created depending on the task. This is a generic one.


### Συνακτικα μοντελα
<h3> OPA2Vec </h3> <br>
Για τα συνακτικά μοντέλα, απαραίτητη είναι η αναπαράσταση των αξιωμάτων σε μορφή text, ώστε στην συνέχεια με την χρήση ενός μοντέλου όπως το Word2Vec, να παραχθούν τα embedding. Στο συγκεκριμένο παράδειγμα, χρησιμοποιούνται και reasoners για να δημιουργηθούν και αξιώματα υποκλάσης, ισοδυναμίας και διάκρησης. Επιπλέον, χρησιμοποιείται άτυπη γνώση όπως μεταδεδομένα οντοτήτων (πχ συνώνημα, ορισμοί) με την χρήση του "extract_and_save_annotation_corpus" της mowl.corpus, όπου είναι και η διαφορά του μοντέλου από το Onto2Vec.


In [ ]:
from mowl.corpus import extract_and_save_axiom_corpus, extract_and_save_annotation_corpus
from mowl.owlapi import OWLAPIAdapter
from mowl.reasoning import MOWLReasoner
from org.semanticweb.elk.owlapi import ElkReasonerFactory
from java.util import HashSet
from mowl.models import SyntacticPlusW2VModel
import os
logging.basicConfig(level=logging.WARNING)
reasoner_factory = ElkReasonerFactory()
reasoner = reasoner_factory.createReasoner(dataset.ontology)
mowl_reasoner = MOWLReasoner(reasoner)

classes = dataset.ontology.getClassesInSignature()
subclass_axioms = mowl_reasoner.infer_subclass_axioms(classes) #αξιωματα υποκλάσεων
equivalent_class_axioms = mowl_reasoner.infer_equivalent_class_axioms(classes) #αξιωματα εξλισωσης κλάσεων

adapter = OWLAPIAdapter()
manager = adapter.owl_manager

axioms = HashSet()
axioms.addAll(subclass_axioms)
axioms.addAll(equivalent_class_axioms)

manager.addAxioms(dataset.ontology, axioms) #προσθηκη των αξιωματων που παρηχθηκαν απο το reasoner στην οντολογία που χρησιμοποιείται

extract_and_save_axiom_corpus(dataset.ontology, "opa2vec_corpus.txt") #αποθηκευση ολων των αξιωματων που υπηρχαν και παράχθηκαν σε μορφη text
extract_and_save_annotation_corpus(dataset.ontology, "opa2vec_corpus.txt", mode="a") #επεκταση text με μεταδεδομένα όπως ορισμοί και περιγραφές στιγμιοτύπων


model = SyntacticPlusW2VModel(dataset, corpus_filepath="opa2vec_corpus.txt") #Word2Vec στο text για παραγωγη embedding
model.set_w2v_model(min_count=1, workers=4, epochs=20)
model.generate_corpus(save=True, with_annotations=True)
model.train()
os.remove("opa2vec_corpus.txt")


INFO:root:Number of inferred axioms: 555583.
INFO:root:Number of inferred axioms: 56751.
INFO:root:Generating axioms corpus
INFO:root:Generating annotation corpus
INFO:gensim.utils:Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=100, alpha=0.025>', 'datetime': '2025-01-03T00:42:19.911182', 'gensim': '4.3.3', 'python': '3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.26100-SP0', 'event': 'created'}
INFO:root:Generating axioms corpus
INFO:root:Generating annotation corpus
INFO:mowl.base_models.syntactic_model:Corpus saved in opa2vec_corpus.txt
INFO:gensim.models.word2vec:collecting all words and their counts
INFO:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #10000, processed 47899 words, keeping 14664 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #20000, processed 95373 words, keeping 24903 word t

### Evaluation του OPA2Vec

In [ ]:
model.set_evaluator(PPIEvaluator)
model.evaluate(dataset.testing)
data4= model.metrics


Class http://4932.YCL020W not found in w2v model
Class http://4932.YCR108C not found in w2v model
Class http://4932.YDR182W-A not found in w2v model
Class http://4932.YDR504C not found in w2v model
Class http://4932.YGL176C not found in w2v model
Class http://4932.YHR078W not found in w2v model
Class http://4932.YJL195C not found in w2v model
Class http://4932.YLR444C not found in w2v model
Class http://4932.YML116W-A not found in w2v model
Class http://www.w3.org/2002/07/owl#Nothing not found in w2v model
Class http://www.w3.org/2002/07/owl#Thing not found in w2v model


### Ευνοϊκή για σύγκριση αναπαράσταση αποδόσεων των μοντέλων με χρήση DataFrame της βιβλιοθήκης pandas.

In [ ]:
import pandas as pd
df = pd.DataFrame([data1, data2, data3,data4], index=['Model 1', 'Model 2', 'Model 3', 'Model 4'])

pd.set_option('display.float_format', lambda x: f'{x:.6f}') #δεκαδική μορφή
print(df)

                 mr      mrr        f_mr    f_mrr      auc    f_auc   hits@1  \
Model 1 1444.729900 0.004990 1444.729900 0.004990 0.760893 0.760893 0.000000   
Model 2 1540.819103 0.004620 1540.819103 0.004620 0.744979 0.744979 0.000000   
Model 3  400.716694 0.026006  400.716694 0.026006 0.933800 0.933800 0.000083   
Model 4 1409.202658 0.004040 1409.202658 0.004040 0.766777 0.766777 0.000664   

          hits@3  hits@10  hits@50  hits@100  f_hits@1  f_hits@3  f_hits@10  \
Model 1 0.001661 0.008056 0.035548  0.074668  0.000000  0.001661   0.008056   
Model 2 0.001910 0.007309 0.036462  0.071346  0.000000  0.001910   0.007309   
Model 3 0.014867 0.063040 0.244435  0.388538  0.000083  0.014867   0.063040   
Model 4 0.002159 0.004900 0.023754  0.045100  0.000664  0.002159   0.004900   

         f_hits@50  f_hits@100  
Model 1   0.035548    0.074668  
Model 2   0.036462    0.071346  
Model 3   0.244435    0.388538  
Model 4   0.023754    0.045100  
